In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tf-nightly-gpu-2.0-preview

In [2]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Convolution2D
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization

from tensorflow.keras import Model

from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import initializers
import tensorflow.keras.backend as K

print(tf.__version__)

2.0.0-dev20190522


In [3]:
# tf.debugging.set_log_device_placement(True)

In [4]:
import handshape_datasets as hd

DATASET_NAME = "lsa16"

data = hd.load(DATASET_NAME)

In [5]:
!pip install -q -U scikit-learn
from sklearn.model_selection import train_test_split

features = data[0]
labels = data[1]['y']

x_train, x_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    test_size=0.33,
                                                    random_state=42)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [7]:
print(train_ds)

<BatchDataset shapes: ((None, 32, 32, 3), (None,)), types: (tf.float64, tf.uint8)>


In [8]:
x_train.shape

(536, 32, 32, 3)

In [9]:
class DenseNet(Model):
    def __init__(self, nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.0, 
                 dropout_rate=0.0, weight_decay=1e-4, classes=1000, batch_size=32):
        super(DenseNet, self).__init__()

        self.eps = 1.1e-5
                
        # compute compression factor
        compression = 1.0 - reduction
        
        self.concat_axis = 3

        # From architecture for ImageNet (Table 1 in the paper)
        nb_filter = 64
        nb_layers = [6,12,24,16] # For DenseNet-121

        self.initial_layers = []
        self.initial_layers.append(ZeroPadding2D((3, 3), name='conv1_zeropadding', 
                                   input_shape=(32, 32, 3), batch_size=batch_size))
        self.initial_layers.append(Convolution2D(nb_filter, 7, 2, name='conv1', use_bias=False))
        self.initial_layers.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name='conv1_bn'))
        self.initial_layers.append(Activation('relu', name='relu1'))
        self.initial_layers.append(ZeroPadding2D((1, 1), name='pool1_zeropadding'))
        self.initial_layers.append(MaxPooling2D((3, 3), strides=(2, 2), name='pool1'))
        
        self.dense_blocks = []
        self.transition_blocks = []

        # Add dense blocks
        for block_idx in range(nb_dense_block - 1):
            stage = block_idx+2
            block, nb_filter = self.dense_block(stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
            self.dense_blocks.append(block)
            
            # Add transition_block
            self.transition_blocks.append(self.transition_block(stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay))
            nb_filter = int(nb_filter * compression)

        final_stage = stage + 1
        block, nb_filter = self.dense_block(final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
        self.dense_blocks.append(block)
        
        self.final_layers = self.final_block(nb_filter, classes)       
    
    def final_block(self, nb_filter, classes):
        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name='conv_final_blk_bn'))
        block.append(Activation('relu', name='relu_final_blk'))
        block.append(GlobalAveragePooling2D(name='pool_final'))
        block.append(Dense(classes, name='fc6'))
        block.append(Activation('softmax', name='prob'))
        return block
    
    def conv_block(self, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
        conv_name_base = 'conv' + str(stage) + '_' + str(branch)
        relu_name_base = 'relu' + str(stage) + '_' + str(branch)

        # 1x1 Convolution (Bottleneck layer)
        inter_channel = nb_filter * 4  
        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_x1_bn'))
        block.append(Activation('relu', name=relu_name_base+'_x1'))
        block.append(Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))

        # 3x3 Convolution
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_x2_bn'))
        block.append(Activation('relu', name=relu_name_base+'_x2'))
        block.append(ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding'))
        block.append(Convolution2D(nb_filter, 3, 1, name=conv_name_base+'_x2', use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))
        return block
                                        

    def dense_block(self, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, 
                    grow_nb_filters=True):
        block = []
        for i in range(nb_layers):
            branch = i+1
            block.append(self.conv_block(stage, branch, growth_rate, dropout_rate, weight_decay))

            if grow_nb_filters:
                nb_filter += growth_rate

        return block, nb_filter
                                        
    def transition_block(self, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
        conv_name_base = 'conv' + str(stage) + '_blk'
        relu_name_base = 'relu' + str(stage) + '_blk'
        pool_name_base = 'pool' + str(stage) 

        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_bn'))
        block.append(Activation('relu', name=relu_name_base))
        block.append(Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))

        block.append(AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base))

        return block

    def call(self, inputs):
        x = inputs
        for layer in self.initial_layers:
            x = layer(x)
        i = 0
        for transition_block in self.transition_blocks:
            concat_feat = x
            for conv_block in self.dense_blocks[i]:                                    
                for layer in conv_block:                                    
                    x = layer(x)
                x = tf.concat([concat_feat, x], self.concat_axis)
                concat_feat = x
            for layer in transition_block:                                    
                x = layer(x)
            i += 1
        concat_feat = x
        for conv_block in self.dense_blocks[i]:                                    
            for layer in conv_block:                                    
                x = layer(x)
            x = tf.concat([concat_feat, x], self.concat_axis)
            concat_feat = x
        for layer in self.final_layers:
            x = layer(x)                                      
        return x

model = DenseNet(classes=16, dropout_rate=0.3)
# model.load_weights(weights_path) 

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


In [11]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [12]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(tf.cast(images, tf.float32))
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


In [13]:
@tf.function
def test_step(images, labels):
    predictions = model(tf.cast(images, tf.float32))
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [14]:
EPOCHS = 400
    
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    if (epoch % 10 == 0):
        template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print (template.format(epoch+1,
                               train_loss.result(),
                               train_accuracy.result()*100,
                               test_loss.result(),
                               test_accuracy.result()*100))
    
    if (epoch % 100 == 0):
        # serialize weights to HDF5
        model.save_weights("model_epoch{}.h5".format(epoch))
        print("Saved model to disk")

Epoch 1, Loss: 2.7782981395721436, Accuracy: 4.664178848266602, Test Loss: 2.776930093765259, Test Accuracy: 3.78787899017334
Saved model to disk
Epoch 11, Loss: 1.4045051336288452, Accuracy: 50.661460876464844, Test Loss: 1.423429250717163, Test Accuracy: 51.61845779418945
Epoch 21, Loss: 0.8005601167678833, Accuracy: 72.08599853515625, Test Loss: 0.8962130546569824, Test Accuracy: 71.24819946289062
Epoch 31, Loss: 0.5714297294616699, Accuracy: 80.15767669677734, Test Loss: 0.7267208695411682, Test Accuracy: 77.87145233154297
Epoch 41, Loss: 0.44438567757606506, Accuracy: 84.63323974609375, Test Loss: 0.6220327019691467, Test Accuracy: 81.59645080566406
Epoch 51, Loss: 0.3576222062110901, Accuracy: 87.63900756835938, Test Loss: 0.5576972365379333, Test Accuracy: 84.36571502685547
Epoch 61, Loss: 0.2990003228187561, Accuracy: 89.66539764404297, Test Loss: 0.5287823677062988, Test Accuracy: 86.18355560302734
Epoch 71, Loss: 0.25688761472702026, Accuracy: 91.12097930908203, Test Loss: 0.

KeyboardInterrupt: 